# 0. Descriptors

- In Python, a descriptor is an object attribute with `binding behavior,` which means that it has methods defined for getting, setting, and deleting the attribute. Descriptors are used to customize attribute access and provide more control over how attributes are managed in classes. 

- The key methods associated with descriptors are:
    - 1. `__get__(self, instance, owner)`: Called when the descriptor's attribute is accessed. Tt should return the value of the attribute. `instance` is the instance of the object that the attribute is accessed on, and `owner` is the class of the instance. 
    - 2. `__set__(self, instance, value)`: Called when the descriptor's attribute is set. It allows you to define the behavior when a value is assigned to the attribute. 
    - 3. `__delete__(self, instance)`: Called when the descriptor's attribute is deleted. It allows you to define the behavior when the attribute is deleted. 

## Example 1: Positive Age

Imagine you want to create a class where the attribute value should always be a non-negative integer such as age assigned to an individual. You can use a descriptor to enforce this constraint. 

In [1]:
class NonNegativeInteger:
    def __init__(self, name) -> None:
        self._name = name

    def __get__(self, instance, owner):
        return instance.__dict__[self._name]

    def __set__(self, instance, value):
        if not isinstance(value, int) or value <0:
            raise ValueError(f"{self._name} must be a non-negative integer")
        instance.__dict__[self._name] = value

class User:
    age = NonNegativeInteger("age")

    def __init__(self, name, age) -> None:
        self.name = name
        self.age = age

In [2]:
user1 = User('Janet', 25)
user1.age

25

In this example, the `User` class has an attribute `age` which is an instance of `NonNegativeInteger`. The descriptor ensures that the age attribute for any User instance is always a non-negative integer. 

In [3]:
try:
    user1.age = -5
except ValueError as e:
    print(e)

age must be a non-negative integer


#  1. Lazily Computed Property

**Purpose of Lazily Computed Property:**
   - Lazily computed properties are used to improve performance by avoiding unnecessary calculations until the property is actually needed.

# 2. Example

The `lazyproperty` decorator allows you to create properties that are computed only once and then cached for future use. Here's a breakdown of the code:


1. **`lazyproperty` Class:**
   - This is a custom class acting as a descriptor.
   - It takes a function (`func`) during initialization, which is assumed to be a method of the class that will be decorated.

2. **`__init__` Method:**
   - The `__init__` method stores the function passed to it (the method to be lazily evaluated).

3. **`__get__` Method:**
   - The `__get__` method is called when the decorated property is accessed.
   - The `if instance is None:` check is necessary because the `__get__` method can also be called when the property is being accessed on the class itself, rather than on an instance of the class. In this case, the instance parameter will be None.
      - If accessed on the class itself (`instance is None`), it returns the `lazyproperty` instance.
      - If accessed on an instance, it computes the value using the stored function and sets the attribute on the instance with the computed value. It then returns the computed value.

4. **Usage in the `Circle` Class:**
   - The `Circle` class has two properties: `area` and `perimeter`, both decorated with `@lazyproperty`.
   - The `area` and `perimeter` properties are computed lazily, meaning that the calculations are performed only when the properties are first accessed, and the results are cached for subsequent accesses.

5. **Example Usage:**
   - Accessing the `area` and `perimeter` properties triggers the lazy computation, and the results are printed.
   - Subsequent accesses to the properties reuse the previously computed values without recalculating.

This approach is a form of memoization, where the results of expensive function calls are cached to avoid redundant computations. It's particularly useful for properties or methods that involve heavy calculations but might not always be needed.

In [4]:
class lazyproperty:
    def __init__(self, func) -> None:
        self.func = func

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance, self.func.__name__, value)
            return value

In [5]:
import math

class Circle:
    def __init__(self, radius) -> None:
        self.radius = radius

    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi * self.radius ** 2

    @lazyproperty
    def perimeter(self):
        print('Computing perimeter')
        return 2 * math.pi * self.radius

In [6]:
circle_instance = Circle(radius=5)

In [7]:
circle_instance.area

Computing area


78.53981633974483

In [8]:
# Accessing the area propety again reuses the previously computed value. No computation
circle_instance.area

78.53981633974483

In [9]:
circle_instance.perimeter

Computing perimeter


31.41592653589793

In [10]:
# Accessing the perimeter propety again reuses the previously computed value. No computation
circle_instance.perimeter

31.41592653589793